# AgentCore Evaluation - 사용 예제

이 노트북은 AgentCore Observability에서 평가하고자 하는 모든 Sessionid를 평가하는 데 도움을 줍니다.

## 설정

In [ ]:
import os
from utils import EvaluationClient  # 평가 클라이언트 유틸리티
import json

# AWS Credentials - Add your credentials here
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'
# os.environ['AWS_ACCESS_KEY_ID'] = ''
# os.environ['AWS_SECRET_ACCESS_KEY'] = ''
# os.environ['AWS_SESSION_TOKEN'] = '' 

## 구성

In [ ]:
# AWS Configuration
REGION = "us-east-1"
AGENT_ID = "strands_claude_eval-YOUR_UNIQUE_ID"
SESSION_ID = "<YOUR_SESSION_ID_HERE>"  # 평가할 세션 ID 직접 지정

# 평가 실험에 대한 메타데이터
metadata = {
        "experiment": "evaluation_test",
        "description": "Testing all evaluator scopes"
    }

## 클라이언트 초기화

In [ ]:
# Initialize evaluation client
eval_client = EvaluationClient(
    region=REGION,
)

## Evaluation 실행

모든 evaluator 유형에 걸쳐 세션을 평가하고 자동으로 대시보드를 생성합니다.

In [ ]:
# Evaluator Groups
# 세션 또는 span 레벨에서 사용 가능한 유연한 evaluator들
FLEXIBLE_EVALUATORS = [
    "Builtin.Correctness",
    "Builtin.Faithfulness",
    "Builtin.Helpfulness",
    "Builtin.ResponseRelevance",
    "Builtin.Conciseness",
    "Builtin.Coherence",
    "Builtin.InstructionFollowing",
    "Builtin.Refusal",
    "Builtin.Harmfulness",
    "Builtin.Stereotyping"
]

# 세션 레벨에서만 사용 가능한 evaluator
SESSION_ONLY_EVALUATORS = ["Builtin.GoalSuccessRate"]

# span 레벨에서만 사용 가능한 evaluator들
SPAN_ONLY_EVALUATORS = [
    "Builtin.ToolSelectionAccuracy",
    "Builtin.ToolParameterAccuracy"
]

In [ ]:
# 각 evaluator 그룹별로 scope 정의
test_groups = [
    {
        "name": "Flexible Evaluators (session scope)",
        "evaluators": FLEXIBLE_EVALUATORS,
        "scope": "session"
    },
    {
        "name": "Session-Only Evaluators",
        "evaluators": SESSION_ONLY_EVALUATORS,
        "scope": "session"
    },
    {
        "name": "Span-Only Evaluators",
        "evaluators": SPAN_ONLY_EVALUATORS,
        "scope": "span"
    }
]

all_results = []

# 각 그룹별로 평가 실행
for group in test_groups:
    try:
        results = eval_client.evaluate_session(
            session_id=SESSION_ID,
            evaluator_ids=group['evaluators'],
            agent_id=AGENT_ID,
            region=REGION,
            scope=group['scope'],
            auto_save_output=True,  # 결과 자동 저장
            auto_create_dashboard=True,  # 대시보드 자동 생성
            metadata=metadata
        )
        
        print(f"\nCompleted: {len(results.results)} evaluations")
        for r in results.results:
            print(f"  {r.evaluator_name}: {r.value} - {r.label}")
            all_results.append(r)
            
    except Exception as e:
        print(f"\nError: {e}")

# 끝